In [2]:
from sumpy.recurrence import _make_sympy_vec, get_reindexed_and_center_origin_on_axis_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)


import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [3]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init_lap, order_lap, recur_laplace = get_reindexed_and_center_origin_on_axis_recurrence(laplace2d)

w = make_identity_diff_op(2)
helmholtz2d = laplacian(w) + w
n_init_helm, order_helm, recur_helmholtz = get_reindexed_and_center_origin_on_axis_recurrence(helmholtz2d)

In [4]:
var = _make_sympy_vec("x", 2)
rct = sp.symbols("r_{ct}")
s = sp.Function("s")
n = sp.symbols("n")

In [5]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs
l_max = 15
derivs_laplace = compute_derivatives(l_max)
derivs_laplace_dict = dict(zip([s(i) for i in range(l_max)], [derivs_laplace[i] for i in range(l_max)]))

In [6]:
def compute_rel_err(nsub):
    return abs((recur_laplace.subs(n, nsub).subs(derivs_laplace_dict).subs(coord_dict) - derivs_laplace[nsub].subs(coord_dict))/derivs_laplace[nsub].subs(coord_dict))

In [24]:
nsub = 14
coord_dict = {var[0]: 0.1 * np.random.rand(), var[1]: np.random.rand()}
recur_coeffs_lap = sp.poly(recur_laplace.subs(n, nsub), [s(i) for i in range(nsub - order_lap, nsub)]).coeffs()
#[i+nsub-order_lap for i in range(len(recur_coeffs_lap))]
coeffs_sub = [(recur_coeffs_lap[i]*derivs_laplace[i+nsub-order_lap]).subs(coord_dict) for i in range(len(recur_coeffs_lap))]
print(coeffs_sub)
ratio = np.abs(coeffs_sub[0]/coeffs_sub[1])
print("ratio: ", ratio, "||||| digits: ",np.log10(float(ratio)), "|||| pred error: 1e-"+str(int(16-np.ceil(np.log10(float(ratio))))))
print("rel. error: ", compute_rel_err(nsub))

[283760210571788., 25680175954536.8, -308530291313573.]
ratio:  11.0497767256013 ||||| digits:  1.0433535026528984 |||| pred error: 1e-14
rel. error:  5.64683791788174e-14


In [8]:
c = sp.symbols("c")

In [12]:
nsub = 5
coord_dict_var = {var[0]: var[0], var[1]: c*var[0]}
recur_coeffs_lap_var = sp.poly(recur_laplace.subs(n, nsub), [s(i) for i in range(nsub - order_lap, nsub)]).coeffs()
#[i+nsub-order_lap for i in range(len(recur_coeffs_lap))]
coeffs_sub_var = [(recur_coeffs_lap_var[i]*derivs_laplace[i+nsub-order_lap]).subs(coord_dict_var) for i in range(len(recur_coeffs_lap_var))]

In [33]:
rat = 100

In [34]:
(coeffs_sub[0]/coeffs_sub[1]).subs(var[0], 1).subs(c, rat).evalf()

-833.361103703457

In [35]:
(coeffs_sub[1]/coeffs_sub[2]).subs(var[0], 1).subs(c, rat).evalf()

0.00120020036408851

In [36]:
(coeffs_sub[0]/coeffs_sub[2]).subs(var[0], 1).subs(c, rat).evalf()

-1.00020030008209